In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import feature_column as fc
from tensorflow.keras.datasets import boston_housing

In [2]:
#download the boston dataset
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [3]:
features = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT']
x_train_df = pd.DataFrame(x_train, columns= features)
x_test_df = pd.DataFrame(x_test, columns= features)
y_train_df = pd.DataFrame(y_train, columns= ['MEDV'])
y_test_df = pd.DataFrame(y_test, columns= ['MEDV'])

In [4]:
#TODO: check correlation of each feature and add to feature_columns list only selected features
feature_columns = []
for feature_name in features:
    feature_columns.append(fc.numeric_column(feature_name, dtype=tf.float32))

In [5]:
#create the input function for the estimator: it returns a tuple (features,labels) in batches)
def estimator_input_fn(df_data, df_label, epochs=10, shuffle = True, batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(df_data),df_label))
        if shuffle:
            ds = ds.shuffle(100)
        ds = ds.batch(batch_size).repeat(epochs)
        return ds
    return input_function

In [6]:
train_input_fn = estimator_input_fn(x_train_df,y_train_df)
val_input_fn = estimator_input_fn(x_test_df,y_test_df,epochs=1,shuffle=False)

In [7]:
#istantiate a LinearRegressor estimator
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns, model_dir="logs/LinearRegression_boston/")
linear_est.train(train_input_fn, steps =100)
result = linear_est.evaluate(val_input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'logs/LinearRegression_boston/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both 

In [8]:
result = linear_est.predict(val_input_fn)
for pred,exp in zip(result,y_test[:32]):
    print("Predicted Value: ", pred['predictions'][0], "Expected:", exp)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from logs/LinearRegression_boston/model.ckpt-400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predicted Value:  3.4354844 Expected: 7.2
Predicted Value:  22.93869 Expected: 18.8
Predicted Value:  21.509516 Expected: 19.0
Predicted Value:  27.159096 Expected: 27.0
Predicted Value:  22.883472 Expected: 22.2
Predicted Value:

In [9]:
#%load_ext tensorboard

In [10]:
#%tensorboard --logdir logs/LinearRegression_boston/